In [1]:
import sys, os
pardir = os.path.abspath(os.path.join(os.path.dirname('__file__'), os.path.pardir))
sys.path.append(pardir)

import numpy as np
import yaafelib
import matplotlib
import matplotlib.pyplot as plt
from collections import namedtuple
from scipy.io import wavfile
from read_labels import read_labels

%matplotlib inline  

In [2]:
path = '/mnt/data/Birdman/16khz//STHELENA-02_20140605_200000_1.wav'
step_size = 512 # no. of samples

rate, signal = wavfile.read(path)
feature_plan = yaafelib.FeaturePlan(sample_rate=rate, normalize=True)
success = feature_plan.loadFeaturePlan('features.config')
engine = yaafelib.Engine()
engine.load(feature_plan.getDataFlow())
feats = engine.processAudio(signal.reshape(1, -1).astype('float64'))

sheets = read_labels('/mnt/data/Birdman/sthelena_labels.xls')
sheet_selected = sheets['STHELENA-02_20140605_200000_1']

In [3]:
print('No. of calls: {}'.format(len(sheet_selected)))
sheet_selected.head()

No. of calls: 52


,Date,File Name,Type of Call,Time Start,Time End,Species,Notes
0,2014-06-05,STHELENA-02_20140605_200000_1,Flight Call,2.905,4.197,Storm Petrel,NaN
1,2014-06-05,STHELENA-02_20140605_200000_1,Flight Call,10.008,11.622,Storm Petrel,NaN
2,2014-06-05,STHELENA-02_20140605_200000_1,Flight Call,14.527,16.464,Storm Petrel,NaN
3,2014-06-05,STHELENA-02_20140605_200000_1,Flight Call,17.110,18.724,Storm Petrel,NaN
4,2014-06-05,STHELENA-02_20140605_200000_1,Flight Call,20.338,21.307,Storm Petrel,NaN


In [4]:
Call = namedtuple('Call', ['start', 'end', 'features'])

feature_lengths = []
storm_petrel_calls = []

for idx, row in sheet_selected.iterrows():
    if row['Species'] == 'Storm Petrel':
        start = int((row['Time Start'] * rate) / step_size)
        end = int((row['Time End'] * rate) / step_size)
        feature_lengths.append(end - start)
        storm_petrel_call_feature = {}
        for name, feature in feats.items():
            storm_petrel_call_feature[name + '_mean'] = feature[start:end].mean(axis=0)
            storm_petrel_call_feature[name + '_std'] = feature[start:end].std(axis=0)
        storm_petrel_calls.append(Call(start, end, storm_petrel_call_feature))

In [9]:
feats['MagnitudeSpectrum'].shape

(35431, 257)